In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time
import pyperclip
import requests
from threading import Thread

In [4]:
# Define the base URL to scrape
URL = 'https://limitlesstcg.com'


# Create a Chrome driver with the options
driver = webdriver.Chrome()

The chromedriver version (117.0.5938.92) detected in PATH at /opt/homebrew/bin/chromedriver might not be compatible with the detected chrome version (118.0.5993.88); currently, chromedriver 118.0.5993.70 is recommended for chrome 118.*, so it is advised to delete the driver in PATH and retry


In [5]:
from functions import get_all_tournaments

filters = ['regional', 'national', 'international', 'worlds', 'special', 'cl', 'rl', 'online', 'players_cup', 'invitational']
tournament_links_by_type = {'regional': [], 'national': [], 'international': [], 'worlds': [], 'others': []}

for filter in filters:
    # Go to URL for all tournaments between 2011 and 2023
    driver.get(URL+f'/tournaments?time=all&type={filter}&format=all&region=all')

    # Get all the links to the tournaments
    tls = get_all_tournaments(driver)
    if filter in ['regional', 'national', 'international', 'worlds']:
        tournament_links_by_type[filter] = tls
    else:
        tournament_links_by_type['others'] += tls

print(f"Regionals: {len(tournament_links_by_type['regional'])}")
print(f"Nationals: {len(tournament_links_by_type['national'])}")
print(f"Internationals: {len(tournament_links_by_type['international'])}")
print(f"Worlds: {len(tournament_links_by_type['worlds'])}")
print(f"Others: {len(tournament_links_by_type['others'])}")

# Close the session
driver.close()

Regionals: 25
Nationals: 23
Internationals: 20
Worlds: 11
Others: 68


In [6]:
with open('data/tournaments.csv', 'w') as f:
        f.write('id_card,name_card,amount_card,price_card,energy_type_card,type_card,combo_type_id,combo_type_name,id_player,name_player,country_player,all_time_score,ranking_player_tournament,id_tournament,category_tournament,name_tournament,country_tournament,year_tournament,month_tournament,day_tournament,valid_rotation_at_tournament\n')

In [7]:
from functions import get_tournament_data

In [8]:
# Iterate through all the categories
for category, tournament_links in tournament_links_by_type.items():
    
    # Iterate through all the tournaments
    for tl in tournament_links:

        # Call the function to get the data from a tournament
        tournament_data = get_tournament_data(category, tl)


The chromedriver version (117.0.5938.92) detected in PATH at /opt/homebrew/bin/chromedriver might not be compatible with the detected chrome version (118.0.5993.88); currently, chromedriver 118.0.5993.70 is recommended for chrome 118.*, so it is advised to delete the driver in PATH and retry
The chromedriver version (117.0.5938.92) detected in PATH at /opt/homebrew/bin/chromedriver might not be compatible with the detected chrome version (118.0.5993.88); currently, chromedriver 118.0.5993.70 is recommended for chrome 118.*, so it is advised to delete the driver in PATH and retry
The chromedriver version (117.0.5938.92) detected in PATH at /opt/homebrew/bin/chromedriver might not be compatible with the detected chrome version (118.0.5993.88); currently, chromedriver 118.0.5993.70 is recommended for chrome 118.*, so it is advised to delete the driver in PATH and retry
The chromedriver version (117.0.5938.92) detected in PATH at /opt/homebrew/bin/chromedriver might not be compatible with 

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"./td[2]/a[1]"}
  (Session info: chrome=118.0.5993.88); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x0000000104cfed98 chromedriver + 4337048
1   chromedriver                        0x0000000104cf6e14 chromedriver + 4304404
2   chromedriver                        0x0000000104923a5c chromedriver + 293468
3   chromedriver                        0x0000000104968d50 chromedriver + 576848
4   chromedriver                        0x000000010495e200 chromedriver + 532992
5   chromedriver                        0x00000001049a3908 chromedriver + 817416
6   chromedriver                        0x000000010495ca5c chromedriver + 526940
7   chromedriver                        0x000000010495d908 chromedriver + 530696
8   chromedriver                